In [1]:
from gspread_dataframe import set_with_dataframe
from oauth2client.service_account import ServiceAccountCredentials
import gspread
import pandas as pd
import numpy as np
import json
import sqlite3
import datetime
from dateutil.relativedelta import relativedelta
import datetime
import os

SPREADSHEET_IDS = {
    "EXA FIRST": "10-B_vV1pvUzXmvGAiHhODGJgCloOsAmqSO9HvXpk_T8",
    "コンサートホールエフ成増": "1EDY2RfjDQNsapVrl2X-UrqPKoXrkQmYJnk3uPqccBxY",
    "第一プラザ坂戸1000": "170MVr-BB3LG-g5ItkDT-8TE6R68RW9zJhRfpvQiy-PE",
    "第一プラザみずほ台店": "1_1722pigi_Z1D6eH0tsPfMneGoS9O09fyqD6F-h1mQA",
    "パールショップともえ川越店": "1i70joJ27Hs7inS-D89z9YMSJO1aRvaBeeWn0n9xpktY",
    "パラッツォ川越店": "179nJF0NvLng7xPKsd_NX2pJBXsDNsO8SJhOvUAvFk2I",
    "第一プラザ狭山店": "1IVb2Woq3n_PDZP87LdW9NpFP3Z6LeyQtONCkx_fWIq4",
    "ニュークラウン川越2号店": "1jocaH94-5GKWUqEqcrybgFcpb1Tn0BhvLMqaADppNWg",
    "TOHO川越店": "1VOr7BNMB-xnHihZrk6MFmetlZ0nCFP86YcrJu9w0dac",
}
# スプレッドシート認証設定
scope = [
    "https://spreadsheets.google.com/feeds",
    "https://www.googleapis.com/auth/drive",
]
jsonf = r"C:\python\dataOnline\anaslo_02\json\spreeadsheet-347321-ff675ab5ccbd.json"
creds = ServiceAccountCredentials.from_json_keyfile_name(jsonf, scope)
client = gspread.authorize(creds)

In [2]:
def create_df_from_database(hall_name, start_date, end_date, model_name=None):
    # Table name 取得
    DB_PATH = r"C:\python\dataOnline\anaslo_02\db\anaslo_02.db"
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
    tables = cursor.fetchall()
    # print(tables)

    cursor.execute(
        "SELECT hall_id, name FROM halls WHERE name LIKE ?", ("%" + hall_name + "%",)
    )
    results = cursor.fetchall()

    # 結果表示
    if results:
        hall_id, hall_name = results[0]
        print(f"🔍 '{hall_name}' を含むホール名が見つかりました。")
    else:
        print(f"❌ '{hall_name}' を含むホール名は見つかりませんでした。")

    query = """
        -- 出玉データにホール名と機種名を結合して取得
        SELECT
            r.*, 
            h.name AS hall_name,
            m.name AS model_name
        FROM results r
        JOIN halls h ON r.hall_id = h.hall_id
        JOIN models m ON r.model_id = m.model_id
        WHERE h.name = ?
        AND r.date BETWEEN ? AND ?
        """

    params = [hall_name, start_date, end_date]
    if model_name:
        query += " AND m.name LIKE ?"
        params.append(f"%{model_name}%")  # 部分一致にする

    query += " ORDER BY r.date DESC, r.unit_no ASC"

    df = pd.read_sql_query(query, conn, params=params)
    print(f"データサイズ: {df.shape[0]} x {df.shape[1]}")
    print(f"📅 検索期間: {start_date} ～ {end_date}", f"📅 抽出期間: {df.date.min()} ～ {df.date.max()}")
    print(f'含まれる日数 : {df["date"].nunique()}')

    return df

In [13]:
# ブドウシミュレーター
def grape_calc_myfive(game, bb, rb, medals, cherry=True):
    bb_medals = 239.25
    rb_medals = 95.25
    replay_rate = 0.411
    if cherry:
        cherry_rate_high = 0.04228
    else:
        cherry_rate_high = 0.05847
    denominator_inner = (
        -medals
        - (
            game * 3
            - (
                bb * bb_medals
                + rb * rb_medals
                + game * replay_rate
                + game * cherry_rate_high
            )
        )
    ) / 8
    grape_rate = (game / denominator_inner) - ((game / denominator_inner) * 2)
    return grape_rate


GRAPE_CONSTANTS = {
    "マイジャグラーV": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.05847,
        "cherryOn": 0.04228,
    },
    "アイムジャグラーEX-TP": {
        "bb": 251.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.06068,
        "cherryOn": 0.040475,
    },
    "ゴーゴージャグラー3": {
        "bb": 239.00,
        "rb": 95.00,
        "replay": 0.411,
        "cherryOff": 0.0661,
        "cherryOn": 0.0372,
    },
    "ファンキージャグラー2": {
        "bb": 239.25,
        "rb": 95.25,
        "replay": 0.411,
        "cherryOff": 0.0603,
        "cherryOn": 0.04324,
    },
}


def calc_grape_rate(row, constants, cherry=True):
    model = row["model_name"]
    if model not in constants:
        return None
    try:
        game = row["game"]
        bb = row["BB"]
        rb = row["RB"]
        medals = row["medals"]
        # 定数取得
        c = constants[model]
        cherry_rate = c["cherryOn"] if cherry else c["cherryOff"]
        # 分母計算式
        denominator = (
            -medals
            - (
                game * 3
                - (
                    bb * c["bb"]
                    + rb * c["rb"]
                    + game * c["replay"]
                    + game * cherry_rate
                )
            )
        ) / 8
        if denominator == 0:
            return None  # ゼロ除算防止
        grape = (game / denominator) - ((game / denominator) * 2)
        return round(grape, 2)

    except Exception as e:
        print(f"⚠️ Grape計算失敗: {model} → {e}")
        return None


def assign_area(unit_no, json_file_path):
    with open(json_file_path, "r", encoding="utf-8") as f:
        area_map = json.load(f)
    for rule in area_map:
        if rule["start"] <= unit_no <= rule["end"]:
            return rule["area"]
    return "other"


def df_preprocessing(df, hall_name):
    json_path = f"C:/python/dataOnline/anaslo_02/json/{hall_name}_area_map.json"
    if not os.path.exists(json_path):
        json_path = f"C:/python/dataOnline/anaslo_02/json/other_area_map.json"
    print(f"データ前処理を行います")
    df_pre = df.copy()
    df_pre["date"] = pd.to_datetime(df_pre["date"])
    df_pre.drop(columns=["result_id", "hall_id", "model_id"], inplace=True)
    df_pre_columns = ["hall_name", "date", "model_name", "unit_no", "game", "BB", "RB", "medals"]
    df_pre = df_pre[df_pre_columns]
    df_pre["BB_rate"] = (df_pre["game"] / df_pre["BB"]).round(1)
    df_pre["RB_rate"] = (df_pre["game"] / df_pre["RB"]).round(1)
    # df_pre["Grape_rate"] = grape_calc_myfive(
    #     df_pre["game"], df_pre["BB"], df_pre["RB"], df_pre["medals"], cherry=True).round(2)
    
    df_pre["Grape_rate"] = df_pre.apply(lambda row: calc_grape_rate(row, GRAPE_CONSTANTS), axis=1)
    df_pre["Total_rate"] = (df_pre["game"] / (df_pre["BB"] + df_pre["RB"])).round(1)
    df_pre["month"] = df_pre["date"].dt.strftime("%Y-%m")
    df_pre["day"] = df_pre["date"].dt.day
    df_pre["weekday"] = df_pre["date"].dt.weekday
    df_pre["year"] = df_pre["date"].dt.year
    df_pre["unit_last"] = df_pre["unit_no"].astype(str).str[-1]

    df_pre["area"] = df_pre["unit_no"].apply(lambda x: assign_area(x, json_path))
    
    df_pre.replace([np.inf, -np.inf], np.nan, inplace=True)
    df_pre = df_pre.fillna(0)

    model_list = list(df["model_name"].unique())
    # for i, model in enumerate(model_list):
    #     print(f"{i+1}: {model}", end=", ")

    return df_pre, model_list


def add_spreadsheet(spreadsheet, sheet_name, df, sheet_clear=False):
    today = datetime.datetime.today()
    rows, cols = df.shape
    try:
        sheet = spreadsheet.worksheet(sheet_name)
        print(f"✅ シート「{sheet_name}」が既に存在します。")
    except gspread.exceptions.WorksheetNotFound:
        sheet = spreadsheet.add_worksheet(
            title=sheet_name, rows=str(rows + 3), cols=str(cols + 3)
        )
        print(f"🆕 シート「{sheet_name}」を新規作成しました。")
    if sheet_clear:
        sheet.clear()
    last_row = len(sheet.get_all_values())
    set_with_dataframe(sheet, df, row=last_row, include_index=True)
    sheet.update_cell(1, 1, today.strftime("%Y-%m-%d UPDATED"))
    print(f"✅ シート '{sheet_name}' に DataFrame を書き込みました！")

## ピボットテーブル作成


In [14]:
def create_pivot_table(
    df,
    index,
    columns,
    pivots=["game", "medals", "BB", "RB"],
    reverse=False,
    margins=True,
    day_target=None,
):
    df_filtered = df.copy()
    if day_target is not None:
        df_filtered = df_filtered[df_filtered["day"] == day_target]

    pivot_results = {}
    for col in pivots:
        table = df_filtered.pivot_table(
            index=index,
            columns=columns,
            values=col,
            aggfunc="sum",
            margins=margins,
            margins_name="total",
        )
        pivot_results[col] = table
        if reverse:
            pivot_results[col] = table.iloc[:, ::-1]

    game = pivot_results["game"]
    medals = pivot_results["medals"]
    rb = pivot_results["RB"]
    bb = pivot_results["BB"]
    rb_rate = (game / rb).round(1)
    total_rate = (game / (bb + rb)).round(1)
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)

    labeled_tables = [
        ("GAME", game),
        ("MEDALS", medals),
        ("RB_RATE", rb_rate),
        ("TOTAL_RATE", total_rate),
        ("MEDAL_RATE", medal_rate),
        ("BB", bb),
        ("RB", rb),
    ]

    # ラベルを MultiIndex に付ける
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合
    interleaved_cols = [
        col
        for pair in zip(
            game.columns,
            medals.columns,
            bb.columns,
            rb.columns,
            medal_rate.columns,
            rb_rate.columns,
            total_rate.columns,
        )
        for col in pair
    ]
    merged = pd.concat([game, medals, medal_rate, bb, rb, rb_rate, total_rate], axis=1)[
        interleaved_cols
    ]
    merged.replace([np.inf, -np.inf, np.nan], None, inplace=True)
    details = {
        "game": game,
        "medals": medals,
        "medal_rate": medal_rate,
        "bb": bb,
        "rb": rb,
        "rb_rate": rb_rate,
        "total_rate": total_rate,
    }

    return merged, details



## 出力データ

- 期間指定
  - 機種別 x 日付
  - 島別 x 日付
  - 台番号 x 日付
  - 島別 x 月
  - 過去 n 日の差枚比較

In [15]:
hall_name = "EXA FIRST"
# hall_name = "コンサートホールエフ成増"
# hall_name = "第一プラザみずほ台店"
# hall_name = "オータ志木駅前店"
hall_name = "第一プラザ坂戸1000"
# hall_name = "ニュークラウン川越2号店"
# hall_name = "TOHO川越店"
# hall_name = "グランドオータ新座駅前店"

model_name = "ジャグラー"

today = datetime.date.today()
start_date = today - relativedelta(months=6, days=today.day - 1)
df_db = create_df_from_database(hall_name, start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
# spreadsheet = client.open_by_key(SPREADSHEET_IDS[hall_name])
# print(f"スプレッドシート: {spreadsheet.title} を開きました。")
df.tail()

🔍 '第一プラザ坂戸1000' を含むホール名が見つかりました。
データサイズ: 16673 x 11
📅 検索期間: 2024-11-01 ～ 2025-05-26 📅 抽出期間: 2024-11-01 ～ 2025-05-24
含まれる日数 : 185
データ前処理を行います


,hall_name,date,model_name,unit_no,game,BB,RB,medals,BB_rate,RB_rate,Grape_rate,Total_rate,month,day,weekday,year,unit_last,area
16668,第一プラザ坂戸1000,2024-11-01,ハッピージャグラーVIII,855,2226,6,3,-1081,371.0,742.0,0.00,247.3,2024-11,1,4,2024,5,h: ハッピージャグラーVIII
16669,第一プラザ坂戸1000,2024-11-01,ハッピージャグラーVIII,856,4088,12,8,-1337,340.7,511.0,0.00,204.4,2024-11,1,4,2024,6,b: アイムジャグラーEX-TP
16670,第一プラザ坂戸1000,2024-11-01,ハッピージャグラーVIII,857,4215,10,12,-1267,421.5,351.2,0.00,191.6,2024-11,1,4,2024,7,b: アイムジャグラーEX-TP
16671,第一プラザ坂戸1000,2024-11-01,ハッピージャグラーVIII,858,4699,20,12,738,235.0,391.6,0.00,146.8,2024-11,1,4,2024,8,b: アイムジャグラーEX-TP
16672,第一プラザ坂戸1000,2024-11-01,アイムジャグラーEX-TP,859,7264,32,20,1314,227.0,363.2,5.88,139.7,2024-11,1,4,2024,9,b: アイムジャグラーEX-TP


## MODEL_RATE


In [16]:

sheet_name = "MODEL_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
model_rate = details["medal_rate"].copy()
model_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, model_rate, sheet_clear=True)
model_rate.head()

MEDAL_RATE                                                   \
day                    1      2      3      4      5      6      7      8   
model_name                                                                  
アイムジャグラーEX-TP      0.993  0.992  0.979  0.978  0.980  0.981  1.006  0.983   
ウルトラミラクルジャグラー      0.981  0.997  0.944  0.963  0.990  1.008  0.977  0.998   
ゴーゴージャグラー3         1.019  0.985  0.985  0.971  1.013  0.985  0.957  0.995   
ジャグラーガールズ          0.968  0.942  1.010  0.959  0.995  0.957  0.980  0.970   
ハッピージャグラーVIII      0.962  0.951  0.950  0.953  0.957  0.997  0.970  0.982   

                             ...                                            \
day                9     10  ...     23     24     25     26     27     28   
model_name                   ...                                             
アイムジャグラーEX-TP  0.981  0.986  ...  0.965  0.988  0.979  0.971  0.962  0.975   
ウルトラミラクルジャグラー  0.957  0.938  ...  0.993  0.991  0.943  1.026  0.982  1.005   
ゴーゴージャグラー3     0.989  0.985  ...  0.971  0.978  0.988  1.000  0.972  0.993   
ジャグラーガールズ      0.971  0.966  ...  0.979  0.954  0.978  0.977  0.981  0.994   
ハッピージャグラーVIII  0.988  0.982  ...  0.991  0.973  1.007  0.968  0.996  1.027   

                                           
day               29     30     31  total  
model_name                                 
アイムジャグラーEX-TP  0.966  0.980  0.991  0.985  
ウルトラミラクルジャグラー  1.026  1.000  0.978  0.988  
ゴーゴージャグラー3     0.996  1.003  0.972  0.992  
ジャグラーガールズ      1.003  0.998  1.003  0.978  
ハッピージャグラーVIII  1.001  0.952  1.026  0.979  

[5 rows x 32 columns]

## ISLAND_RATE


In [17]:
sheet_name = f"ISLAND_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["area"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
island_rate = details["medal_rate"].copy()
island_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, island_rate, sheet_clear=True)
island_rate.head()

MEDAL_RATE                                                   \
day                       1      2      3      4      5      6      7      8   
area                                                                           
a: アイムジャグラーEX-TP      0.993  1.021  1.011  0.919  0.881  1.060  0.814  0.992   
b: アイムジャグラーEX-TP      0.986  0.980  0.979  0.957  1.003  0.981  0.991  1.005   
c: アイムジャグラーEX-TP      0.998  0.985  0.945  0.993  0.979  0.977  1.006  0.966   
d: マイジャグラーV           0.992  0.983  0.988  0.981  0.979  0.986  0.972  0.985   
e: ファンキージャグラー2        0.985  0.991  0.975  0.963  0.972  0.998  0.965  0.995   

                                ...                                            \
day                   9     10  ...     23     24     25     26     27     28   
area                            ...                                             
a: アイムジャグラーEX-TP  0.936  0.849  ...  0.997  0.945  0.909  0.964  0.893  0.981   
b: アイムジャグラーEX-TP  0.974  0.965  ...  0.980  0.971  0.943  0.956  0.973  0.977   
c: アイムジャグラーEX-TP  1.000  0.990  ...  0.967  0.995  1.006  0.996  0.979  0.984   
d: マイジャグラーV       1.000  0.986  ...  0.984  0.994  0.973  0.978  0.994  0.980   
e: ファンキージャグラー2    0.939  0.964  ...  0.943  0.989  0.966  1.026  0.976  0.978   

                                              
day                  29     30     31  total  
area                                          
a: アイムジャグラーEX-TP  0.920  0.971  1.054  0.983  
b: アイムジャグラーEX-TP  0.949  0.969  0.955  0.976  
c: アイムジャグラーEX-TP  0.960  0.982  1.007  0.990  
d: マイジャグラーV       0.976  0.979  0.988  0.986  
e: ファンキージャグラー2    0.977  0.981  0.985  0.979  

[5 rows x 32 columns]

## UNIT_RATE


In [18]:
sheet_name = f"UNIT_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name", "unit_no"]
columns = ["day"]
merged, details = create_pivot_table(df, index, columns, reverse=False, margins=True)
target_rate = 1.05
unit_rate = details["medal_rate"].copy()
unit_rate[("MEDAL_RATE", f"{target_rate}+")] = (
    unit_rate.iloc[:, :-1] >= target_rate
).sum(axis=1)
countif = (unit_rate.iloc[:-1, :] >= target_rate).sum(axis=0)
unit_rate = pd.concat(
    [unit_rate, pd.DataFrame([countif], index=[(f"count_{target_rate}+", "")])], axis=0
)
unit_rate.replace([np.inf, -np.inf], np.nan, inplace=True)
unit_rate = unit_rate[~unit_rate.iloc[:, 1].isna()]
unit_rate.to_csv(csv_path)
# add_spreadsheet(spreadsheet, sheet_name, unit_rate, sheet_clear=True)
unit_rate.head()

MEDAL_RATE                                                   \
day                        1      2      3      4      5      6      7      8   
アイムジャグラーEX-TP 663      1.003  1.024  1.042  0.936  0.927  0.955  0.557  0.910   
              664      0.950  1.010  1.065  0.786  0.886  0.989  0.417  1.028   
              665      0.925  1.110  0.859  1.018  0.857  1.235 -0.333  0.942   
              666      1.020  0.847  0.983  0.918  0.726  0.930  0.872  1.065   
              809      1.064  1.080  0.897  0.959  0.919  0.919  0.900  0.916   

                                 ...                                     \
day                    9     10  ...     24     25     26     27     28   
アイムジャグラーEX-TP 663  0.926  0.848  ...  0.941  0.815  0.969  0.960  1.020   
              664  1.096  0.913  ...  1.009  0.965  1.013  0.897  0.787   
              665  0.874  0.887  ...  1.043  1.015  1.011  0.643  0.913   
              666  0.938  0.664  ...  0.712  0.818  0.631  0.840  1.034   
              809  0.923  0.989  ...  1.026  0.702  1.043  0.920  0.901   

                                                     
day                   29     30     31  total 1.05+  
アイムジャグラーEX-TP 663  0.920  1.068  1.080  0.981     5  
              664  0.821  1.032  0.850  0.973     3  
              665  1.002  0.507  0.576  1.025     8  
              666  0.904  0.640    NaN  0.957     3  
              809  0.732  0.952  0.918  0.978     4  

[5 rows x 33 columns]

# DAYn

- SHEET_NAME : DAY1，DAY2，… DAY31


In [19]:
index = ["area", "unit_no"]
columns = ["date"]
day_list = [25]
# for day_target in range(today.day - 1, today.day + 1):
for day_target in day_list:
    sheet_name = f"DAY{day_target}"
    csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
    merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True, day_target=day_target)
    merged.to_csv(csv_path)
    # add_spreadsheet(spreadsheet, f"DAY{day_target}", merged, sheet_clear=True,)
merged.head()

GAME MEDALS    BB    RB MEDAL_RATE RB_RATE  \
date                     total  total total total      total   total   
area             unit_no                                               
a: アイムジャグラーEX-TP 663      2063  -1146     4     4      0.815   515.8   
                 664      2007   -213     8     3      0.965   669.0   
                 665      1335     59     5     4      1.015   333.8   
                 666       639   -348     1     1      0.818   639.0   
b: アイムジャグラーEX-TP 856      4347  -3059     7     8      0.765   543.4   

                         TOTAL_RATE                GAME              MEDALS  \
date                          total 2025-04-25 00:00:00 2025-04-25 00:00:00   
area             unit_no                                                      
a: アイムジャグラーEX-TP 663          257.9               932.0              -401.0   
                 664          182.5               957.0               -87.0   
                 665          148.3              1105.0               343.0   
                 666          319.5               321.0              -122.0   
b: アイムジャグラーEX-TP 856          289.8               804.0              -669.0   

                                          BB  ...          MEDAL_RATE  \
date                     2025-04-25 00:00:00  ... 2024-12-25 00:00:00   
area             unit_no                      ...                       
a: アイムジャグラーEX-TP 663                     2.0  ...                None   
                 664                     4.0  ...                None   
                 665                     5.0  ...                None   
                 666                     1.0  ...                None   
b: アイムジャグラーEX-TP 856                     1.0  ...               0.596   

                                     RB_RATE          TOTAL_RATE  \
date                     2024-12-25 00:00:00 2024-12-25 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                    None                None   
                 664                    None                None   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                   312.0               312.0   

                                        GAME              MEDALS  \
date                     2024-11-25 00:00:00 2024-11-25 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                   218.0              -297.0   
                 664                    None                None   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                   597.0              -169.0   

                                          BB                  RB  \
date                     2024-11-25 00:00:00 2024-11-25 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                     0.0                 0.0   
                 664                    None                None   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                     2.0                 1.0   

                                  MEDAL_RATE             RB_RATE  \
date                     2024-11-25 00:00:00 2024-11-25 00:00:00   
area             unit_no                                           
a: アイムジャグラーEX-TP 663                   0.546                None   
                 664                    None                None   
                 665                    None                None   
                 666                    None                None   
b: アイムジャグラーEX-TP 856                   0.906               597.0   

                                  TOTAL_RATE  
date

## MONTH_RATE

In [20]:
start_date = today - relativedelta(months=100, days=today.day-1)
df_db = create_df_from_database("EXA FIRST", start_date, today, model_name=model_name)
df, model_list = df_preprocessing(df_db, hall_name)
sheet_name = "MONTH_RATE"
csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
index = ["model_name"]
columns = ["month"]
merged, details = create_pivot_table(df, index, columns, reverse=True, margins=True)
month_rate = details["medal_rate"].copy()
# add_spreadsheet(spreadsheet, sheet_name, month_rate, sheet_clear=True)
month_rate.to_csv(csv_path)
month_rate

🔍 'EXA FIRST' を含むホール名が見つかりました。
データサイズ: 63384 x 11
📅 検索期間: 2017-01-01 ～ 2025-05-26 📅 抽出期間: 2024-04-01 ～ 2025-05-24
含まれる日数 : 417
データ前処理を行います


MEDAL_RATE                                                  \
month              total 2025-05 2025-04 2025-03 2025-02 2025-01 2024-12   
model_name                                                                 
アイムジャグラーEX-TP      1.002   0.996   0.998   1.001   0.993   0.997   1.000   
ウルトラミラクルジャグラー      0.998   0.988   0.993   0.997   1.002   1.000   1.001   
ゴーゴージャグラー3         1.002   1.000   0.993   0.992   1.001   1.000   0.996   
ジャグラーガールズ          1.003   0.999   0.993   1.001   0.990   0.996   0.994   
ハッピージャグラーVIII      0.995   0.992   0.995     NaN     NaN     NaN     NaN   
ファンキージャグラー2        0.996   0.991   0.992   0.993   0.996   0.990   0.995   
マイジャグラーV           1.010   1.003   1.000   1.002   1.003   1.006   1.005   
ミスタージャグラー          1.001   0.998   1.002   0.991   0.996   0.990   1.006   
total              1.006   0.999   0.998   0.999   1.001   1.002   1.002   

                                                                               
month         2024-11 2024-10 2024-09 2024-08 2024-07 2024-06 2024-05 2024-04  
model_name                                                                     
アイムジャグラーEX-TP   1.001   0.999   1.001   1.000   1.007   1.005   1.004   1.008  
ウルトラミラクルジャグラー     NaN     NaN     NaN     NaN     NaN     NaN     NaN     NaN  
ゴーゴージャグラー3      1.005   0.995   0.998   1.003   1.003   1.006   1.010   1.012  
ジャグラーガールズ       0.995   0.999   1.003   0.997   1.005   1.005   1.010   1.016  
ハッピージャグラーVIII   0.957   1.004     NaN     NaN     NaN     NaN     NaN     NaN  
ファンキージャグラー2     0.996   1.000   0.989   1.001   0.986   0.990   1.000   1.007  
マイジャグラーV        1.002   1.008   1.009   1.012   1.011   1.017   1.021   1.025  
ミスタージャグラー       0.997   1.002   1.008   1.007   1.009     NaN     NaN     NaN  
total           1.001   1.004   1.005   1.007   1.008   1.011   1.015   1.018

## HISTORY

In [25]:
start_date = datetime.date(today.year, today.month-1, 1) - relativedelta(days=6)
df_db = create_df_from_database(hall_name, start_date, today, model_name="ジャグラー")
df, model_list = df_preprocessing(df_db, hall_name)


pivot_targets = ["medals", "game", "RB_rate", "Total_rate", "Grape_rate"]
index_targets = ["area", "model_name", "unit_no"]
columns_targets = ["date"]

merged_by_model = pd.DataFrame()
for model in df["model_name"].unique():
    
    # モデルごとにデータをフィルタリング
    df_model = df[df["model_name"] == model]
    pivot_results = {}
    for col in pivot_targets:
        table = df_model.pivot_table(
            index=index_targets,
            columns=columns_targets,
            values=col,
            aggfunc="sum",
        )
        pivot_results[col] = table
    
    medals = pivot_results["medals"]
    game = pivot_results["game"]
    rb_rate = pivot_results["RB_rate"]
    total_rate = pivot_results["Total_rate"]
    grape_rate = pivot_results["Grape_rate"]
    medal_rate = ((medals + game * 3) / (game * 3)).round(3)
    
    # ランキング列作成
    rolling7 = medals.T.rolling(7, min_periods=7).sum().T
    medal_rank7 = rolling7.rank(method="min", ascending=True)
    medal_rank7 = medal_rank7.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling5 = medals.T.rolling(5, min_periods=5).sum().T
    medal_rank5 = rolling5.rank(method="min", ascending=True)
    medal_rank5 = medal_rank5.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    rolling3 = medals.T.rolling(3, min_periods=3).sum().T
    medal_rank3 = rolling3.rank(method="min", ascending=True)
    medal_rank3 = medal_rank3.fillna(0).replace([np.inf, -np.inf], 0).astype(int)
    medal_rank1 = medals.rank(method="min", ascending=True)
    medal_rank1 = medal_rank1.fillna(0).replace([np.inf, -np.inf], 0).astype(int)

    # MultiIndex化（ラベル付け）
    labeled_tables = [
        ("GRAPE_RATE", grape_rate),
        ("TOTAL_RATE", total_rate),
        ("RB_RATE", rb_rate),
        ("GAME", game),
        ("RATE_MEDAL", medal_rate),
        ("MEDALS", medals),
        ("3ROLLING", rolling3),
        ("5ROLLING", rolling5),
        ("7ROLLING", rolling7),
        ("1RANK", medal_rank1),
        ("3RANK", medal_rank3),
        ("5RANK", medal_rank5),
        ("7RANK", medal_rank7),
    ]
    for label, df_table in labeled_tables:
        df_table.columns = pd.MultiIndex.from_product([[label], df_table.columns])

    # 列を交互に整列して統合・NaN除去・日付ソート・一部データ削除
    interleaved_cols = [
        col
        for col_group in zip(*(df.columns for _, df in labeled_tables))
        for col in col_group
    ]
    merged = pd.concat([df for _, df in labeled_tables], axis=1)[interleaved_cols]
    merged = merged.iloc[:, ::-1]
    # display(merged)
    merged = merged[~merged.iloc[:, 7].isna()]

    # エリアごとに空行挿入して整形
    merged_by_area = pd.DataFrame()
    for area in merged.index.get_level_values("area").unique():
        area_merged = merged.xs(area, level="area", drop_level=False)
        if not area_merged.empty:
            empty_index = pd.MultiIndex.from_tuples(
                [("", " ", " ")], names=merged.index.names
            )
            empty_row = pd.DataFrame(
                [[""] * area_merged.shape[1]],
                index=empty_index,
                columns=area_merged.columns,
            )
            merged_by_area = pd.concat([merged_by_area, area_merged, empty_row])
            
    merged_by_area.to_csv("test.csv")

    # インデックス削除
    merged_by_area = merged_by_area.droplevel("area")
    
    if not merged_by_area.empty:
        empty_index = pd.MultiIndex.from_tuples(
            [(" ", " ")], names=merged_by_area.index.names
        )
        empty_row = pd.DataFrame(
            [[""] * merged_by_area.shape[1]],
            index=empty_index,
            columns=merged_by_area.columns,
        )
        merged_by_model = pd.concat(
            [merged_by_model, merged_by_area, empty_row], axis=0
        )

# 累計した最後の7日を削除
merged_by_model = merged_by_model.iloc[:, :-len(labeled_tables)*6]
merged_by_model.replace([np.inf, -np.inf], np.nan, inplace=True)

csv_path = f"C:/python/dataOnline/anaslo_02/out/{hall_name}_{sheet_name}.csv"
merged_by_model.to_csv(csv_path)

sheet_name = "HISTORY"
# add_spreadsheet(spreadsheet, sheet_name, merged_by_model, sheet_clear=True)

merged_by_model.head()

🔍 '第一プラザ坂戸1000' を含むホール名が見つかりました。
データサイズ: 5292 x 11
📅 検索期間: 2025-03-26 ～ 2025-05-26 📅 抽出期間: 2025-03-26 ～ 2025-05-24
含まれる日数 : 54
データ前処理を行います


7RANK      5RANK      3RANK      1RANK   7ROLLING  \
date                2025-05-24 2025-05-24 2025-05-24 2025-05-24 2025-05-24   
model_name  unit_no                                                          
ファンキージャグラー2 751              0          0          3          3        NaN   
            752              0          0          4          4        NaN   
            753              0          0          5          5        NaN   
            754              0          0          6          6        NaN   
            755              0          0          2          2        NaN   

                      5ROLLING   3ROLLING     MEDALS RATE_MEDAL       GAME  \
date                2025-05-24 2025-05-24 2025-05-24 2025-05-24 2025-05-24   
model_name  unit_no                                                          
ファンキージャグラー2 751            NaN      453.0     1128.0      1.063     5992.0   
            752            NaN     1813.0     1523.0       1.09     5644.0   
            753            NaN     2117.0     1698.0      1.085     6655.0   
            754            NaN     2471.0     1779.0       1.08     7386.0   
            755            NaN      134.0      134.0      1.007     6147.0   

                     ...      1RANK   7ROLLING   5ROLLING   3ROLLING  \
date                 ... 2025-04-01 2025-04-01 2025-04-01 2025-04-01   
model_name  unit_no  ...                                               
ファンキージャグラー2 751      ...          0        NaN        NaN        NaN   
            752      ...          0        NaN        NaN        NaN   
            753      ...          0        NaN        NaN        NaN   
            754      ...          0        NaN        NaN        NaN   
            755      ...          0        NaN        NaN        NaN   

                        MEDALS RATE_MEDAL       GAME    RB_RATE TOTAL_RATE  \
date                2025-04-01 2025-04-01 2025-04-01 2025-04-01 2025-04-01   
model_name  unit_no                                                          
ファンキージャグラー2 751            NaN        NaN        NaN        NaN        NaN   
            752            NaN        NaN        NaN        NaN        NaN   
            753            NaN        NaN        NaN        NaN        NaN   
            754            NaN        NaN        NaN        NaN        NaN   
            755            NaN        NaN        NaN        NaN        NaN   

                    GRAPE_RATE  
date                2025-04-01  
model_name  unit_no             
ファンキージャグラー2 751            NaN  
            752            NaN  
            753            NaN  
            754            NaN  
            755            NaN  

[5 rows x 624 columns]